In [ ]:
# set module
import pandas as pd


from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os

new_dir = "/content/drive/MyDrive/L2CS-Net"
os.chdir(new_dir)  # 현재 작업 디렉토리 변경
current_dir = os.getcwd()
print("현재 작업 디렉토리:", current_dir)

현재 작업 디렉토리: /content/drive/MyDrive/L2CS-Net


In [ ]:
!pwd

/content/drive/MyDrive/L2CS-Net


In [ ]:
## 패키지 설치
! pip install -r requirements.txt  


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install git+https://github.com/elliottzheng/face-detection.git@master


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/elliottzheng/face-detection.git (to revision master) to /tmp/pip-req-build-k28wbvgs
  Running command git clone --filter=blob:none --quiet https://github.com/elliottzheng/face-detection.git /tmp/pip-req-build-k28wbvgs
  Resolved https://github.com/elliottzheng/face-detection.git to commit 786fbab7095623c348e251f1f0a8b323721c6a84
  Preparing metadata (setup.py) ... done
  Created wheel for face-detection: filename=face_detection-1.0.5-py3-none-any.whl size=1638438 sha256=fdcb013a17ea518eb3d96c6792e36f19eb9fd9b4da557c230187937a99aaaa87
  Stored in directory: /tmp/pip-ephem-wheel-cache-m4q4t3ow/wheels/08/ac/4b/4d71660ae29737385e9967e87f576cc4ef7e1ba06d244dfbb3
Successfully built face-detection


In [ ]:
# 스크립트 활
# !python demo_video.py \
#  --snapshot models/L2CSNet_gaze360.pkl \
#  --gpu 0

In [ ]:
def calculate_euclidean_distance(point1, point2):
    # 점의 개수가 같은지 확인
    if len(point1) != len(point2):
        raise ValueError("The dimensions of the points do not match.")

    # 각 차원별로 제곱 차이를 계산하여 더함
    squared_diff = sum((coord1 - coord2)**2 for coord1, coord2 in zip(point1, point2))
    
    # 제곱 차이의 합에 루트를 씌워 유클리드 거리 계산
    distance = math.sqrt(squared_diff)
    return distance

In [ ]:
## 모듈 다운로드:

import numpy as np
from main import get_gaze_predictions
import numpy as np
from scipy.interpolate import CubicSpline
import matplotlib.pyplot as plt
import math

snapshot_path = 'models/L2CSNet_gaze360.pkl'
arch = 'ResNet50'
gpu_id = '0'


##  bad_data test 코드
video_files = [f for f in os.listdir("./datasets/bad_data/") if f.endswith('.mp4')]
for video_file in video_files:
    video_name = video_file.split('/')[-1].split('.')[0]  # Extracting video name from video path
    video_file_path = './datasets/bad_data/' + video_file


    try:
      # get_gaze_predictions 함수 호출하여 pitch_predicted_list와 yaw_predicted_list 얻기
      pitch_predicted_list, yaw_predicted_list, imputation = get_gaze_predictions(video_file_path, snapshot_path, arch, gpu_id)

      # 유클리드 거리 계산
      distance_list = []
      for i in range(len(pitch_predicted_list) - 1):
        point1 = [pitch_predicted_list[i], yaw_predicted_list[i]]
        point2 = [pitch_predicted_list[i+1], yaw_predicted_list[i+1]]
        
        distance = calculate_euclidean_distance(point1, point2)
        distance_list.append(distance)

      try:
        x = list(range(len(distance_list)))
        y = distance_list

        cs = CubicSpline(x, y)
        interpolated_x = np.linspace(min(x), max(x), num=imputation)
        interpolated_y = cs(interpolated_x)

        # 원본 데이터와 보간한 데이터를 순서에 맞게 삽입하여 새로운 리스트 생성
        new_x = list(range(len(distance_list)+imputation))
        new_y = [] ## 보간된 유클리드 거리
        i = 0
        for j in range(len(x)):
            new_y.append(y[j])
            if i < len(interpolated_x) and x[j] >= interpolated_x[i]:
                new_y.append(interpolated_y[i])
                i += 1

        # 값들을 파일에 저장
        np.save("./gaze_vector/eye_moving/bad_data/eye_gaze_moving_list_" + video_name +".npy", new_y)
      except: 
        np.save("./gaze_vector/eye_moving/bad_data/eye_gaze_moving_list_" + video_name +".npy", y)


    except:
      print("error")
      print(video_file_path)
      print(video_name)
 





Loading snapshot.
Loading snapshot.
Loading snapshot.
error
./datasets/bad_data/1DmNV9C1hbY.mp4
1DmNV9C1hbY
Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
error
./datasets/bad_data/Bfr499ggo-0.mp4
Bfr499ggo-0
Loading snapshot.
Loading snapshot.
Loading snapshot.
error
./datasets/bad_data/bOL9jKpeJRs.mp4
bOL9jKpeJRs
Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
error
./datasets/bad_data/cXypl4FnoZo.mp4
cXypl4FnoZo
Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
error
./datasets/bad_data/f9O3YtZ2VfI.mp4
f9O3

In [ ]:
## 에러코드 처리 -> 위 원본 코드에 합침
import numpy as np
from main import get_gaze_predictions
import math

# 비디오 파일 리스트
# video_paths = ['./datasets/bad_test1.mp4', './datasets/bad_test2.mp4', './datasets/bad_test3.mp4']
video_paths = ['./datasets/bad_data/1DmNV9C1hbY.mp4', 
'./datasets/bad_data/Bfr499ggo-0.mp4',
'./datasets/bad_data/bOL9jKpeJRs.mp4',
'./datasets/bad_data/cXypl4FnoZo.mp4',
'./datasets/bad_data/f9O3YtZ2VfI.mp4',
'./datasets/bad_data/G-xst2euQUc.mp4',
'./datasets/bad_data/9T9Hf74oK10.mp4']

snapshot_path = 'models/L2CSNet_gaze360.pkl'
arch = 'ResNet50'
gpu_id = '0'

for video_path in video_paths:
    # get_gaze_predictions 함수 호출하여 pitch_predicted_list와 yaw_predicted_list 얻기
    pitch_predicted_list, yaw_predicted_list, imputation = get_gaze_predictions(video_path, snapshot_path, arch, gpu_id)
    video_name = video_path.split('/')[-1].split('.')[0]  # Extracting video name from video path

    # 값들을 파일에 저장
    # np.save("pitch_predicted_list.npy", pitch_predicted_list)
    # np.save("yaw_predicted_list.npy", yaw_predicted_list)

    distance_list = []
    for i in range(len(pitch_predicted_list) - 1):
        point1 = [pitch_predicted_list[i], yaw_predicted_list[i]]
        point2 = [pitch_predicted_list[i+1], yaw_predicted_list[i+1]]
        
        distance = calculate_euclidean_distance(point1, point2)
        distance_list.append(distance)

    x = list(range(len(distance_list)))
    y = distance_list

    # cs = CubicSpline(x, y)
    # interpolated_x = np.linspace(min(x), max(x), num=imputation)
    # interpolated_y = cs(interpolated_x)

    # # 원본 데이터와 보간한 데이터를 순서에 맞게 삽입하여 새로운 리스트 생성
    # new_x = list(range(len(distance_list)+imputation))
    # new_y = []
    # i = 0
    # for j in range(len(x)):
    #     new_y.append(y[j])
    #     if i < len(interpolated_x) and x[j] >= interpolated_x[i]:
    #         new_y.append(interpolated_y[i])
    #         i += 1

    np.save("./gaze_vector/eye_moving/bad_data/eye_gaze_moving_list_" + video_name +".npy", y)




Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
Loading snapshot.
